# Rolling Linear Regression (no leakage)
### *“Simplicity is the ultimate sophistication.” – Leonardo da Vinci*

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction

train_path = "../input/jpx-tokyo-stock-exchange-prediction/train_files/"
df = pd.read_csv(train_path+"stock_prices.csv")

env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

df = df[df["Date"]>=df.Date.unique()[-1]] # using only the last day from train set

for (prices, _, _, _, _, sample_prediction) in iter_test:
    cur_date = prices.Date.iat[0]
    cods = prices.SecuritiesCode.unique()
    df = pd.concat([df, prices])
    df = df.sort_values(["SecuritiesCode", "Date"])
    df.ExpectedDividend.fillna(0,inplace=True)
    df.ffill(inplace=True)
    tgts = {}
    for cod in cods:
        tr = df[df.SecuritiesCode==cod]
        z = np.polyfit([0,1], tr.Close[-2:] , 1) # LR with last two points
        p = np.poly1d(z) # linear function to predict
        tgts[cod] = p(3)/p(2) - 1 # Calc next Target
    tr = df[df.Date==cur_date].copy()
    tr.Target=tr["SecuritiesCode"].map(tgts) 
    tr.loc[:,"Rank"]=tr.groupby("Date")["Target"].rank(ascending=False, 
                                                       method="first").astype(int)-1 
    pred = tr.set_index("SecuritiesCode")["Rank"]
    sample_prediction['Rank'] = sample_prediction["SecuritiesCode"].map(pred)
    env.predict(sample_prediction)